# TensorFlow2

### Using Tensorflow like Numpy

In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [4]:
tensor1 = tf.constant([[1.,2.,3.],[4.,5.,6.]])

In [5]:
print(tensor1)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


In [6]:
tensor1.shape

TensorShape([2, 3])

In [7]:
tensor1.dtype

tf.float32

In [11]:
tensor1[:,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [13]:
tensor1 @ tf.transpose(tensor1)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [14]:
a = np.array([2.,4.,5.])
tensor2 = tf.constant(a) # numpy to tensor

In [15]:
tensor2

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [16]:
b = tensor2.numpy() # tensor to numpy

In [17]:
b

array([2., 4., 5.])

In [18]:
tf.square(a) # numpy에 텐서플로의 연산 적용!

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [19]:
np.square(tensor2)

array([ 4., 16., 25.])

In [21]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [22]:
t2 = tf.constant(40., dtype=tf.float64)

In [23]:
tf.constant(2.0) + tf.cast(t2, dtype=tf.float32) # use tf.cast to change datatypes!

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [24]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])

In [25]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [26]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [28]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [30]:
v[:,2].assign([0.,2.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  2.]], dtype=float32)>

In [31]:
v.scatter_nd_update(indices=[[0,0],[1,2]], updates=[100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

## 사용자 정의모델과 훈련 

In [34]:
def huber_fn(y_true, y_pred): #작으면 관대하고 크면 절댓값
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1 # mask array!
    squared_loss = tf.square(error)
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [35]:
huber_fn(tf.constant([1.,2.,3.,4.]), tf.constant([0.5, 0.5, 1,1]))

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.25, 1.  , 1.5 , 2.5 ], dtype=float32)>

* To Use Our Own Loss Function
```
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train)
```

#### 시용자 정의 요소를 가진 모델을 저장하고 로드하기

```
model = keras.models.load_model("my_model_with_custom_loss.h5", 
        custom_objects={"huber_fn": huber_fn})
```

In [36]:
def create_huber(threshold=1.0): # a function that returnes a function with given paramiters
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold # mask array!
        squared_loss = tf.square(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn


```
model.compile(loss = create_huber(2.0), optimizer="nadam")
```

```
model = keras.models.load_model("my_model_with_custom_loss.h5", 
        custom_objects={"huber_fn": create_huber(2.0)})
```

In [39]:
from tensorflow import keras

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kargs):
        self.threshold=threshold
        super.__init__(**kargs)
        pass
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold # mask array!
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold":self.threshold}

``` 
model.compile(loss=HuberLoss(2.0), optimizer="nadam")


model = keras.load_model("my_model_with_custum_loss_class.h5",
                          custum_objects={"Huberloss": HuberLoss}) #class까지 들어가진 X
```

## 활성화함수, 초기화, 규제, 제한을 커스터마이징

In [40]:
def my_softplus(z): # activation function
    return tf.math.log(tf.exp(z) + 1.0)

In [41]:
def my_glorot_initializer(shape, dtype=tf.float32): # weight initialization
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

In [42]:
def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [43]:
def my_positive_weights(weights): # kernel_constraint -> forces weights to be positive
    return tf.where(weights < 0., tf.zeros_like(weights), weights) # 두개의 동일한 크기를 가지고 있는 행렬에서 골라씀!

In [44]:
layer = keras.layers.Dense(30, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

In [45]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

#### 사용자 정의 지표

```
model.compile(loss="mse", optimizer="nadam", metrics=[create_huber(2.0)]
```

In [46]:
precision = keras.metrics.Precision()
precision([0,1,1,1,0,1,0,1],[1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [48]:
precision([0,0,0,0],[1,1,1,1]) #예측값이 기록됨!

<tf.Tensor: shape=(), dtype=float32, numpy=0.30769232>

In [53]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [54]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [52]:
precision.reset_states()

#### 결과가 저장되어 진짜 모델의 평균을 구할 수 있는 HuberMetric 클래스

In [55]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs): # kwargs로 기본 매개변수 처리
        super().__init__(**kargs) #기본 매기변수 처리(예, dtype)
        self.threshold = threshold
        self.huber_fn = create_huber(threshold)
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return { **base_config, "threshold": self.threshold}

## 사용자 정의 층

In [56]:
#가중치가 필요 없는 층
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

In [57]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        pass
    
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name = "kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initalizer="zeros")
        super.build(batch_input_shape)
        
    def call(self, X):
        return self.activation( X @ self.kernel + self.bias)
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units":self.units, "activation": keras.activations.serialize(self.activation)}

In [58]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X # input이 tuple로 들어옴!
        return [X1 + X2, X1*X2, X1/X2]
    def compute_output(self, batch_input_shape):
        b1, b2 = batch_input_shape #shape도 tuple로 들어옴!
        return [b1, b1, b1]

In [59]:
# call에 training 매개변수 존재!
class MyGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
    def call(self, X, training=None): # training params in call!
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

## 사용자 정의 모델 

In [60]:
class ResidualBlock(keras.layers.Layer): #이걸 상속해야함!
    def __init__(self, n_layers, n_neurons, **kargs):
        super().__init__(**kargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") for _ in range(n_layers)] # n_layer개수의 dense layer!
        
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

In [61]:
class ResidualRegressor(keras.Model): #subclassing API
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)
        pass
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1+3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

### 모델 구성요소에 기반한 손실과 지표

In [62]:
class ReconstructingRegressor(keras.Model): #재구성 오차를 가지고 있는 모델
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal") for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        pass
    
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs) #보조출력을 위한 layer -> 실재로 흘러가기 전에는 input shape를 모름으로 흘릴때 build 호출됨!
        super().build(batch_input_shape)
        pass
    
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_meanu(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        return self.out(Z)

## 자동미분을 사용하여 그래디언트 계산

In [63]:
def f1(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

In [65]:
#수치적 미분
w1, w2 = 5, 3
eps = 1e-6

print((f1(w1+eps, w2) - f1(w1, w2)) / eps)
print((f1(w1, w2+eps) - f1(w1, w2)) / eps)

36.000003007075065
10.000000003174137


In [67]:
#자동미분
w1, w2 = tf.Variable(5.), tf.Variable(3.)

with tf.GradientTape() as tape:
    z = f1(w1, w2)

gradients = tape.gradient(z, [w1,w2])

In [68]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [71]:
with tf.GradientTape(persistent=True) as tape:
    z = f1(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)

print(dz_dw1, dz_dw2)
del tape #계속 남아있으므로 지워줘야함!

tf.Tensor(36.0, shape=(), dtype=float32) tf.Tensor(10.0, shape=(), dtype=float32)


In [74]:
c1, c2 = tf.constant(5.), tf.constant(3.)

with tf.GradientTape() as tape:
    z = f1(c1, c2)

print(tape.gradient(z, [c1, c2]))

[None, None]


In [75]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f1(c1, c2)

print(tape.gradient(z, [c1, c2]))

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


In [76]:
def f1(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

In [77]:
with tf.GradientTape(persistent=True) as tape:
    z = f1(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2)

print(dz_dw1, dz_dw2)
del tape #계속 남아있으므로 지워줘야함!

tf.Tensor(30.0, shape=(), dtype=float32) None


In [78]:
x = tf.Variable([300.])
with tf.GradientTape() as tape:
    z = my_softplus(x)

gradients = tape.gradient(z, [x])

In [79]:
gradients

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([nan], dtype=float32)>]

In [85]:
@tf.custom_gradient ########꼭 붙이기!!!
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / ( 1+ 1/exp)
    return tf.math.log(exp), my_softplus_gradients #뒤에 리턴되는건 custom_gradient에 들어감!

In [86]:
x = tf.Variable([300.])
with tf.GradientTape() as tape:
    z = my_better_softplus(x)

gradients = tape.gradient(z, [x])

In [87]:
gradients

[<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>]

## 사용자 정의 훈련 반복

In [88]:
# 이동호 하이 ## 누구야 ㅋㅋㅋㅋㅋㅋㅋㅋ

In [105]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal", kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg),
])

In [106]:
#훈련세트에서 샘플배치 추출
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size) #이렇게 해서 인덱스 추출
    return X[idx], y[idx]

In [117]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = "-".join(["{}: {:.4f}".format(m.name, m.result()) for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [108]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [109]:
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

In [110]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2) 

In [112]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error # a function
mean_loss = keras.metrics.Mean() # a recorder!
metrics = [keras.metrics.MeanAbsoluteError()]

In [120]:
for epoch in range(1, n_epochs + 1):
    print("\nepoch {}/{}".format(epoch, n_epochs))
    for step in range(n_steps):
        X_batch, y_batch = random_batch(X_train, y_train)
        with tf.GradientTape() as tape: # use this to record gradients!!
            y_pred = model(X_batch,training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables)) # apply gradient
        
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        
        mean_loss(loss) # record loss!
        
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step*batch_size, len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()


epoch 1/5
47968/48000 - mean: 8.2666-mean_absolute_error: 2.4848
epoch 2/5
47968/48000 - mean: 8.3599-mean_absolute_error: 2.4973
epoch 3/5
47968/48000 - mean: 8.3487-mean_absolute_error: 2.4961
epoch 4/5
47968/48000 - mean: 8.2790-mean_absolute_error: 2.4908
epoch 5/5
47968/48000 - mean: 8.3410-mean_absolute_error: 2.4955

## functions and Graphs in Tensorflow

In [121]:
def cube(x):
    return x**3

In [122]:
cube(tf.constant([2.]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([8.], dtype=float32)>

In [123]:
tf_cube = tf.function(cube) #function into tf-function!

In [124]:
tf_cube

In [125]:
tf_cube(2.)

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [126]:
@tf.function
def tf_cube(x):
    return x**3

In [127]:
tf_cube(3)

<tf.Tensor: shape=(), dtype=int32, numpy=27>

In [128]:
tf.autograph.to_code(tf_cube.python_function)

"def tf__tf_cube(x):\n    with ag__.FunctionScope('tf_cube', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:\n        do_return = False\n        retval_ = ag__.UndefinedReturnValue()\n        try:\n            do_return = True\n            retval_ = ag__.ld(x) ** 3\n        except:\n            do_return = False\n            raise\n        return fscope.ret(retval_, do_return)\n"